In [ ]:
#TimeSeriesClustering_ChangeFinder_100data_BeforAfterChange1sec
#検知したデータを用いてDTW-kmeans(100個)
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
from pylab import rcParams
from tslearn.clustering import TimeSeriesKMeans
from datetime import datetime
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import changefinder
from sklearn.model_selection import train_test_split,StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier


with tf.device('/device:GPU:0'): #GPU使用
    
    def make_all_ppg_array(data,original_data):
        time=0
        for line in original_data:

            if time < 12000 :
                data[time] = int(line.split()[3])#何列目とってくるか
                time += 1
                
        return data


    def Anomaly_Detection(ppg,answer,label):

        data =np.zeros((12000),int)#データ格納用
        det_data = np.zeros((0,det_data_len),int)#検知データ格納用
        sub_det_data = np.zeros((det_data_len),int) #検知データ作成用


        data = make_all_ppg_array(data,ppg)

        #とりあえず描画
        plt.plot(data, linestyle = "dashed" , label="acc")
        plt.xlabel('Epochs',fontsize = 18)
        plt.ylabel('Accuracy',fontsize = 18)
        plt.legend()
        plt.tick_params()
        plt.show()

        cf = changefinder.ChangeFinder(r=0.01, order=1, smooth=7)

        score = np.zeros((12000),int)
        score_num = 0
        for i in data:
            sub_score = cf.update(i)
            score[score_num] = sub_score
            score_num += 1

        
        det_time=0
        det_data_num=0
        wait_F = 0
        for t in range(0,len(score)):
            if score[t] > 60 and wait_F == 0 and t>2000:
                print(f"record:{t}")
                for k in range(t-100,t+100):
                    sub_det_data[det_time] = data[k]
                    det_time += 1
                det_time = 0
                det_data = np.r_[det_data,sub_det_data.reshape(1,-1)]
                answer_list.append(answer)
                label_list.append(label)
                wait_F = 1
            elif wait_F > 0 :
                wait_F += 1
                if wait_F > 501 :
                    wait_F = 0

        print(det_data)



        # プロット
        fig = plt.figure()
        ax = fig.add_subplot(111)
        ax.plot(score)
        ax2 = ax.twinx()
        ax2.plot(data,'r')
        plt.show()
        
        return det_data

    det_data_len = 200
    get_data = np.zeros((10,det_data_len),int)
    answer_list = []
    label_list = []
    
    with open("gu_sampling100ms_relax7s_act3s_2byte.txt") as ppg:
        get_data = Anomaly_Detection(ppg,"gu",0)
    
    with open("choki_sampling100ms_relax7s_act3s_2byte.txt") as ppg:
        get_data= np.concatenate([get_data,Anomaly_Detection(ppg,"choki",1)],0)
        
    with open("par_sampling100ms_relax7s_act3s_2byte.txt") as ppg:
        get_data= np.concatenate([get_data,Anomaly_Detection(ppg,"par",2)],0)
        
    with open("onerev_sampling100ms_relax7s_act3s_2byte.txt") as ppg:
        get_data= np.concatenate([get_data,Anomaly_Detection(ppg,"one",3)],0)
        
    with open("three_sampling100ms_relax7s_act3s_2byte.txt") as ppg:
        get_data= np.concatenate([get_data,Anomaly_Detection(ppg,"three",4)],0)
        
    with open("four_sampling100ms_relax7s_act3s_2byte.txt") as ppg:
        get_data= np.concatenate([get_data,Anomaly_Detection(ppg,"four",5)],0)
        
    with open("fox_sampling100ms_relax7s_act3s_2byte.txt") as ppg:
        get_data= np.concatenate([get_data,Anomaly_Detection(ppg,"fox",6)],0)
        
    with open("phone_sampling100ms_relax7s_act3s_2byte.txt") as ppg:
        get_data= np.concatenate([get_data,Anomaly_Detection(ppg,"phone",7)],0)
        
    with open("goodrev_sampling100ms_relax7s_act3s_2byte.txt") as ppg:
        get_data= np.concatenate([get_data,Anomaly_Detection(ppg,"good",8)],0)
        
    with open("koyubirev_sampling100ms_relax7s_act3s_2byte.txt") as ppg:
        get_data= np.concatenate([get_data,Anomaly_Detection(ppg,"koyubi",9)],0)
    
    #k分割交差検証
    kf = StratifiedKFold(n_splits=5, shuffle=True)

    for train_index, eval_index in kf.split(get_data,label_list):
        X_train, X_test = get_data[train_index], get_data[eval_index]
        Y_train, Y_test = label_list[train_index], label_list[eval_index]
        
        accuracy_list = []
        sns.set()
        k_range = range(1, 10)
        for k in k_range:
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_train, Y_train)
            Y_pred = knn.predict(X_test)
            accuracy_list.append(metrics.accuracy_score(Y_test, Y_pred))
            print(confusion_matrix(Y_test,Y_pred)) #混同行列を作成

        figure = plt.figure()
        ax = figure.add_subplot(111)
        ax.plot(k_range, accuracy_list)
        ax.set_xlabel('k-nn')
        ax.set_ylabel('accuracy')
        plt.show()
